# Microscope Simulation with SPIDS

This notebook demonstrates how to simulate microscope imaging systems using SPIDS's multi-instrument framework.

## Key Features
- High numerical aperture (NA) optical systems
- Multiple illumination modes (brightfield, darkfield, phase contrast, DIC)
- 3D point spread function (PSF) computation
- Physically accurate propagation using Angular Spectrum method
- Proper sampling validation (Nyquist criterion)

## 1. Setup and Import

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from mpl_toolkits.axes_grid1 import make_axes_locatable

from prism.core.instruments import Microscope, MicroscopeConfig, create_instrument


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 2. Oil Immersion Microscope Configuration

Let's configure a high-NA oil immersion microscope, typical of advanced fluorescence microscopy systems.

In [ ]:
# Configure oil immersion microscope (NA=1.4)
config_oil = MicroscopeConfig(
    numerical_aperture=1.4,
    magnification=100,
    wavelength=532e-9,  # Green laser (532nm)
    medium_index=1.515,  # Oil immersion medium
    n_pixels=512,
    pixel_size=6.5e-6,  # Camera pixel size
)

# Create microscope
microscope_oil = Microscope(config_oil)

# Display microscope information
info = microscope_oil.get_info()
print("Oil Immersion Microscope Configuration:")
print("=" * 40)
for key, value in info.items():
    print(f"{key:<20}: {value}")

# Calculate theoretical resolution
resolution_nm = microscope_oil.resolution_limit * 1e9
object_pixel_nm = (config_oil.pixel_size / config_oil.magnification) * 1e9
print(f"\nTheoretical resolution (Abbe limit): {resolution_nm:.1f} nm")
print(f"Object-space pixel size: {object_pixel_nm:.1f} nm")
print(f"Sampling factor: {resolution_nm / (2 * object_pixel_nm):.2f}x Nyquist")

## 3. Point Spread Function (PSF) Computation

The PSF characterizes the microscope's response to a point source.

In [ ]:
# Compute 2D PSF
psf_2d = microscope_oil.compute_psf()

# Convert to numpy for plotting
psf_2d_np = psf_2d.cpu().numpy()

# Plot PSF with multiple views
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 2D PSF image
ax = axes[0, 0]
im = ax.imshow(
    psf_2d_np,
    cmap="hot",
    extent=[
        -256 * object_pixel_nm,
        256 * object_pixel_nm,
        -256 * object_pixel_nm,
        256 * object_pixel_nm,
    ],
)
ax.set_title("2D PSF - Linear Scale")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Position (nm)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Log scale PSF
ax = axes[0, 1]
im = ax.imshow(
    np.log10(psf_2d_np + 1e-6),
    cmap="hot",
    extent=[
        -256 * object_pixel_nm,
        256 * object_pixel_nm,
        -256 * object_pixel_nm,
        256 * object_pixel_nm,
    ],
)
ax.set_title("2D PSF - Log Scale")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Position (nm)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax, label="log10(Intensity)")

# Zoomed central region
center = psf_2d.shape[0] // 2
zoom_size = 50
ax = axes[0, 2]
psf_zoom = psf_2d_np[
    center - zoom_size : center + zoom_size, center - zoom_size : center + zoom_size
]
im = ax.imshow(
    psf_zoom,
    cmap="hot",
    extent=[
        -zoom_size * object_pixel_nm,
        zoom_size * object_pixel_nm,
        -zoom_size * object_pixel_nm,
        zoom_size * object_pixel_nm,
    ],
)
ax.set_title("PSF Central Region")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Position (nm)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Horizontal cross-section
ax = axes[1, 0]
x_nm = np.arange(-256, 256) * object_pixel_nm
ax.plot(x_nm, psf_2d_np[center, :], "b-", linewidth=2)
ax.axhline(y=0.5, color="r", linestyle="--", alpha=0.5, label="FWHM level")
ax.set_title("PSF Horizontal Cross-section")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Normalized Intensity")
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_xlim([-1000, 1000])

# Vertical cross-section
ax = axes[1, 1]
ax.plot(x_nm, psf_2d_np[:, center], "g-", linewidth=2)
ax.axhline(y=0.5, color="r", linestyle="--", alpha=0.5, label="FWHM level")
ax.set_title("PSF Vertical Cross-section")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Normalized Intensity")
ax.grid(True, alpha=0.3)
ax.legend()
ax.set_xlim([-1000, 1000])

# Radial average
ax = axes[1, 2]
y, x = np.ogrid[: psf_2d.shape[0], : psf_2d.shape[1]]
r = np.sqrt((x - center) ** 2 + (y - center) ** 2).astype(int)
radial_mean = np.bincount(r.ravel(), psf_2d_np.ravel()) / np.bincount(r.ravel())
r_nm = np.arange(len(radial_mean)) * object_pixel_nm
ax.plot(r_nm[:100], radial_mean[:100], "k-", linewidth=2)
ax.axhline(y=0.5, color="r", linestyle="--", alpha=0.5, label="FWHM level")
ax.axvline(x=resolution_nm / 2, color="b", linestyle="--", alpha=0.5, label="Abbe limit/2")
ax.set_title("PSF Radial Average")
ax.set_xlabel("Radius (nm)")
ax.set_ylabel("Normalized Intensity")
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

# Calculate FWHM
half_max = 0.5
indices = np.where(psf_2d_np[center, :] >= half_max)[0]
if len(indices) > 0:
    fwhm_pixels = indices[-1] - indices[0]
    fwhm_nm = fwhm_pixels * object_pixel_nm
    print(f"\nMeasured FWHM: {fwhm_nm:.1f} nm")
    print(f"Theoretical resolution (0.61λ/NA): {resolution_nm:.1f} nm")
    print(f"Ratio (FWHM/Theory): {fwhm_nm / resolution_nm:.2f}")

## 4. 3D PSF Computation

For 3D imaging applications, we need to understand the axial (z) resolution as well.

In [ ]:
# Compute 3D PSF with 41 z-slices
print("Computing 3D PSF (this may take a moment)...")
psf_3d = microscope_oil.compute_psf(z_slices=41)
psf_3d_np = psf_3d.cpu().numpy()

# Calculate depth of field
dof = 2 * config_oil.wavelength * config_oil.medium_index / (config_oil.numerical_aperture**2)
dof_nm = dof * 1e9
z_range_nm = dof_nm
z_positions = np.linspace(-z_range_nm / 2, z_range_nm / 2, 41)

print(f"3D PSF shape: {psf_3d.shape}")
print(f"Axial range: ±{z_range_nm / 2:.0f} nm")
print(f"Theoretical DOF: {dof_nm:.0f} nm")

# Visualize 3D PSF
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# XY slice at focus (z=0)
ax = axes[0, 0]
im = ax.imshow(psf_3d_np[20, :, :], cmap="hot")
ax.set_title("XY Slice at Focus (z=0)")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# XZ slice through center
ax = axes[0, 1]
im = ax.imshow(
    psf_3d_np[:, 256, :],
    cmap="hot",
    aspect="auto",
    extent=[-256 * object_pixel_nm, 256 * object_pixel_nm, z_range_nm / 2, -z_range_nm / 2],
)
ax.set_title("XZ Slice (Axial View)")
ax.set_xlabel("X Position (nm)")
ax.set_ylabel("Z Position (nm)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# YZ slice through center
ax = axes[0, 2]
im = ax.imshow(
    psf_3d_np[:, :, 256],
    cmap="hot",
    aspect="auto",
    extent=[-256 * object_pixel_nm, 256 * object_pixel_nm, z_range_nm / 2, -z_range_nm / 2],
)
ax.set_title("YZ Slice (Axial View)")
ax.set_xlabel("Y Position (nm)")
ax.set_ylabel("Z Position (nm)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Maximum intensity projection (MIP) along Z
ax = axes[1, 0]
mip_z = np.max(psf_3d_np, axis=0)
im = ax.imshow(mip_z, cmap="hot")
ax.set_title("Maximum Intensity Projection (Z)")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Axial intensity profile
ax = axes[1, 1]
axial_profile = psf_3d_np[:, 256, 256]
ax.plot(z_positions, axial_profile, "b-", linewidth=2)
ax.axhline(y=0.5, color="r", linestyle="--", alpha=0.5, label="FWHM level")
ax.set_title("Axial Intensity Profile")
ax.set_xlabel("Z Position (nm)")
ax.set_ylabel("Normalized Intensity")
ax.grid(True, alpha=0.3)
ax.legend()

# 3D PSF at different z-positions
ax = axes[1, 2]
z_indices = [0, 10, 20, 30, 40]
colors = ["blue", "green", "red", "orange", "purple"]
for idx, color in zip(z_indices, colors):
    z_nm = z_positions[idx]
    radial = psf_3d_np[idx, 256, :]
    ax.plot(x_nm, radial, color=color, alpha=0.7, label=f"z={z_nm:.0f}nm")
ax.set_title("PSF at Different Z Positions")
ax.set_xlabel("X Position (nm)")
ax.set_ylabel("Normalized Intensity")
ax.set_xlim([-1000, 1000])
ax.grid(True, alpha=0.3)
ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

# Calculate axial FWHM
indices_z = np.where(axial_profile >= 0.5)[0]
if len(indices_z) > 0:
    fwhm_z = z_positions[indices_z[-1]] - z_positions[indices_z[0]]
    print(f"\nMeasured axial FWHM: {fwhm_z:.0f} nm")
    print(f"Theoretical DOF: {dof_nm:.0f} nm")
    print(f"Ratio (FWHM_z/DOF): {fwhm_z / dof_nm:.2f}")

## 5. Different Illumination Modes

Microscopes support various illumination modes for different imaging applications.

In [ ]:
# Create a simple test sample - circular beads
def create_bead_sample(size, bead_positions, bead_radius=10):
    """Create a sample with circular beads."""
    sample = torch.zeros((size, size), dtype=torch.complex64)
    y, x = torch.meshgrid(torch.arange(size), torch.arange(size), indexing="ij")

    for pos in bead_positions:
        dist = torch.sqrt((x - pos[0]) ** 2 + (y - pos[1]) ** 2)
        sample[dist <= bead_radius] = 1.0

    return sample


# Create sample with multiple beads
sample = create_bead_sample(512, [(100, 100), (200, 300), (350, 150), (400, 400)], bead_radius=15)

# Image with different illumination modes
illumination_modes = ["brightfield", "darkfield", "phase", "dic"]
images = {}

for mode in illumination_modes:
    print(f"Imaging with {mode} illumination...")
    image = microscope_oil.forward(sample, illumination_mode=mode)
    images[mode] = image.cpu().numpy()

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original sample
ax = axes[0, 0]
ax.imshow(torch.abs(sample).numpy(), cmap="gray")
ax.set_title("Original Sample")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.axis("off")

# Brightfield
ax = axes[0, 1]
im = ax.imshow(images["brightfield"], cmap="gray")
ax.set_title("Brightfield Imaging")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.axis("off")

# Darkfield
ax = axes[0, 2]
im = ax.imshow(images["darkfield"], cmap="hot")
ax.set_title("Darkfield Imaging")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.axis("off")

# Phase contrast
ax = axes[1, 0]
im = ax.imshow(images["phase"], cmap="gray")
ax.set_title("Phase Contrast")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.axis("off")

# DIC
ax = axes[1, 1]
im = ax.imshow(images["dic"], cmap="gray")
ax.set_title("Differential Interference Contrast (DIC)")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.axis("off")

# Comparison plot - line profiles
ax = axes[1, 2]
y_line = 100  # Line position for profile
for mode, color in zip(illumination_modes, ["blue", "red", "green", "orange"]):
    profile = images[mode][y_line, :]
    ax.plot(profile, color=color, alpha=0.7, label=mode.capitalize())
ax.set_title(f"Line Profiles at Y={y_line}")
ax.set_xlabel("X Position (pixels)")
ax.set_ylabel("Intensity")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics for each mode
print("\nImage Statistics:")
print("=" * 50)
for mode in illumination_modes:
    img = images[mode]
    print(
        f"{mode.capitalize():15} - Min: {img.min():.3f}, Max: {img.max():.3f}, "
        f"Mean: {img.mean():.3f}, Std: {img.std():.3f}"
    )

## 6. Comparing Different Microscope Configurations

Let's compare oil immersion vs. air objectives.

In [ ]:
# Configure air objective microscope (NA=0.9)
config_air = MicroscopeConfig(
    numerical_aperture=0.9,
    magnification=40,
    wavelength=532e-9,
    medium_index=1.0,  # Air
    n_pixels=512,
    pixel_size=4.5e-6,  # Smaller pixel for proper Nyquist sampling
)

# Create air objective microscope
microscope_air = Microscope(config_air)

# Compute PSFs for comparison
psf_oil = microscope_oil.compute_psf().cpu().numpy()
psf_air = microscope_air.compute_psf().cpu().numpy()

# Calculate resolutions
res_oil = microscope_oil.resolution_limit * 1e9
res_air = microscope_air.resolution_limit * 1e9

# Plot comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Oil immersion PSF
ax = axes[0, 0]
center = 256
zoom = 100
im = ax.imshow(
    psf_oil[center - zoom : center + zoom, center - zoom : center + zoom],
    cmap="hot",
    extent=[-zoom * 65, zoom * 65, -zoom * 65, zoom * 65],
)
ax.set_title(f"Oil Immersion (NA={config_oil.numerical_aperture})")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Position (nm)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Air objective PSF
ax = axes[0, 1]
object_pixel_air = config_air.pixel_size / config_air.magnification * 1e9
im = ax.imshow(
    psf_air[center - zoom : center + zoom, center - zoom : center + zoom],
    cmap="hot",
    extent=[
        -zoom * object_pixel_air,
        zoom * object_pixel_air,
        -zoom * object_pixel_air,
        zoom * object_pixel_air,
    ],
)
ax.set_title(f"Air Objective (NA={config_air.numerical_aperture})")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Position (nm)")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Cross-section comparison
ax = axes[0, 2]
x_oil = (np.arange(512) - 256) * 65  # nm
x_air = (np.arange(512) - 256) * object_pixel_air  # nm
ax.plot(
    x_oil, psf_oil[256, :], "b-", linewidth=2, label=f"Oil (NA={config_oil.numerical_aperture})"
)
ax.plot(
    x_air, psf_air[256, :], "r-", linewidth=2, label=f"Air (NA={config_air.numerical_aperture})"
)
ax.set_xlim([-1000, 1000])
ax.set_title("PSF Cross-sections")
ax.set_xlabel("Position (nm)")
ax.set_ylabel("Normalized Intensity")
ax.legend()
ax.grid(True, alpha=0.3)

# Resolution comparison bar chart
ax = axes[1, 0]
objectives = ["Oil NA=1.4", "Air NA=0.9"]
resolutions = [res_oil, res_air]
colors = ["blue", "red"]
bars = ax.bar(objectives, resolutions, color=colors, alpha=0.7)
ax.set_ylabel("Resolution (nm)")
ax.set_title("Theoretical Resolution Comparison")
for bar, res in zip(bars, resolutions):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 5,
        f"{res:.0f} nm",
        ha="center",
        va="bottom",
    )
ax.grid(True, axis="y", alpha=0.3)

# Image simulation with both microscopes
# Create a resolution target
resolution_target = torch.zeros((512, 512), dtype=torch.complex64)
# Add line pairs with different spacings
spacings_nm = [200, 300, 400, 500, 600]
for i, spacing in enumerate(spacings_nm):
    spacing_pixels = int(spacing / 65)  # Convert to pixels (65nm/pixel for oil)
    y_start = i * 100 + 50
    for x in range(100, 400, spacing_pixels):
        resolution_target[y_start : y_start + 20, x : x + spacing_pixels // 3] = 1.0

# Image with both microscopes
image_oil = microscope_oil.forward(resolution_target).cpu().numpy()
image_air = microscope_air.forward(resolution_target).cpu().numpy()

# Plot imaging comparison
ax = axes[1, 1]
ax.imshow(image_oil, cmap="gray")
ax.set_title("Oil Immersion Imaging")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.axis("off")

ax = axes[1, 2]
ax.imshow(image_air, cmap="gray")
ax.set_title("Air Objective Imaging")
ax.set_xlabel("X (pixels)")
ax.set_ylabel("Y (pixels)")
ax.axis("off")

plt.tight_layout()
plt.show()

# Print comparison summary
print("\nMicroscope Comparison Summary:")
print("=" * 50)
print(f"{'Parameter':<25} {'Oil Immersion':<20} {'Air Objective':<20}")
print("-" * 65)
print(
    f"{'Numerical Aperture':<25} {config_oil.numerical_aperture:<20} {config_air.numerical_aperture:<20}"
)
print(f"{'Medium Index':<25} {config_oil.medium_index:<20} {config_air.medium_index:<20}")
print(f"{'Magnification':<25} {config_oil.magnification:<20} {config_air.magnification:<20}")
print(f"{'Resolution (nm)':<25} {res_oil:<20.1f} {res_air:<20.1f}")
print(f"{'Improvement Factor':<25} {'-':<20} {f'{res_air / res_oil:.2f}x worse':<20}")

## 7. Noise Modeling in Microscopy

Real microscope images include various noise sources.

In [ ]:
# Create a sample with varying intensity
sample_intensity = torch.zeros((512, 512), dtype=torch.complex64)
# Add regions with different intensities
sample_intensity[100:200, 100:200] = 0.2  # Dim region
sample_intensity[100:200, 300:400] = 0.5  # Medium region
sample_intensity[300:400, 100:200] = 1.0  # Bright region
sample_intensity[300:400, 300:400] = 0.8  # Medium-bright region

# Image with and without noise
image_clean = microscope_oil.forward(sample_intensity, add_noise=False)
image_noisy = microscope_oil.forward(sample_intensity, add_noise=True)

# Multiple noisy realizations for statistics
n_realizations = 10
noisy_stack = torch.stack(
    [microscope_oil.forward(sample_intensity, add_noise=True) for _ in range(n_realizations)]
)
noise_std = torch.std(noisy_stack, dim=0)

# Plot results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Clean image
ax = axes[0, 0]
im = ax.imshow(image_clean.cpu().numpy(), cmap="gray")
ax.set_title("Clean Image (No Noise)")
ax.axis("off")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Noisy image
ax = axes[0, 1]
im = ax.imshow(image_noisy.cpu().numpy(), cmap="gray")
ax.set_title("Noisy Image")
ax.axis("off")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Noise map (difference)
ax = axes[0, 2]
noise = (image_noisy - image_clean).cpu().numpy()
im = ax.imshow(noise, cmap="RdBu", vmin=-0.05, vmax=0.05)
ax.set_title("Noise Map (Noisy - Clean)")
ax.axis("off")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Standard deviation map
ax = axes[1, 0]
im = ax.imshow(noise_std.cpu().numpy(), cmap="hot")
ax.set_title(f"Noise Std Dev ({n_realizations} samples)")
ax.axis("off")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# SNR map
ax = axes[1, 1]
snr = image_clean / (noise_std + 1e-10)
im = ax.imshow(snr.cpu().numpy(), cmap="viridis", vmin=0, vmax=50)
ax.set_title("Signal-to-Noise Ratio")
ax.axis("off")
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)

# Histogram of pixel values
ax = axes[1, 2]
ax.hist(
    image_clean.cpu().numpy().flatten(),
    bins=50,
    alpha=0.5,
    label="Clean",
    color="blue",
    density=True,
)
ax.hist(
    image_noisy.cpu().numpy().flatten(),
    bins=50,
    alpha=0.5,
    label="Noisy",
    color="red",
    density=True,
)
ax.set_title("Pixel Value Distribution")
ax.set_xlabel("Intensity")
ax.set_ylabel("Density")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate and print noise statistics
print("\nNoise Statistics:")
print("=" * 50)
print(f"Mean noise: {noise.mean():.6f}")
print(f"Std noise: {noise.std():.6f}")
print(f"Min noise: {noise.min():.6f}")
print(f"Max noise: {noise.max():.6f}")
print(f"\nAverage SNR in bright region: {snr[300:400, 100:200].mean().item():.1f}")
print(f"Average SNR in dim region: {snr[100:200, 100:200].mean().item():.1f}")

## 8. Factory Function Usage

The instrument factory provides a unified interface for creating different instrument types.

In [ ]:
# Use factory function to create microscope

# Create microscope using factory
# Note: pixel_size must satisfy Nyquist criterion for the given NA/wavelength/magnification
config = MicroscopeConfig(
    numerical_aperture=1.2,
    magnification=60,
    wavelength=488e-9,  # Blue laser
    medium_index=1.33,  # Water immersion
    n_pixels=256,
    pixel_size=5.5e-6,  # Smaller pixel size to satisfy Nyquist
)

microscope = create_instrument(config)

# Verify it's a microscope
print(f"Instrument type: {microscope.get_instrument_type()}")
print(f"Resolution limit: {microscope.resolution_limit * 1e9:.1f} nm")

# Calculate Nyquist criterion for reference
nyquist_limit = 0.5 * config.wavelength / config.numerical_aperture
object_pixel = config.pixel_size / config.magnification
print("\nSampling verification:")
print(f"  Nyquist limit: {nyquist_limit * 1e9:.1f} nm")
print(f"  Object pixel size: {object_pixel * 1e9:.1f} nm")
print(f"  Sampling OK: {object_pixel < nyquist_limit}")

# The factory ensures proper validation
try:
    # This should fail - NA > medium_index
    bad_config = MicroscopeConfig(
        numerical_aperture=1.5,
        medium_index=1.0,  # Air
        magnification=100,
        wavelength=532e-9,
        n_pixels=256,
        pixel_size=6.5e-6,
    )
    bad_microscope = create_instrument(bad_config)
except ValueError as e:
    print(f"\nValidation error caught: {e}")

print("\nFactory function properly validates configurations!")

## Summary

This notebook demonstrated:

1. **Microscope Configuration**: Setting up oil immersion and air objective microscopes
2. **PSF Computation**: 2D and 3D point spread functions with physical accuracy
3. **Resolution Analysis**: Comparing theoretical (Abbe limit) vs. measured resolution
4. **Illumination Modes**: Brightfield, darkfield, phase contrast, and DIC imaging
5. **Objective Comparison**: Oil immersion vs. air objectives performance
6. **Noise Modeling**: Realistic noise addition and SNR analysis
7. **Factory Pattern**: Using the unified instrument creation interface

Key takeaways:
- Higher NA provides better resolution (oil > air)
- Different illumination modes reveal different sample features
- 3D PSF shows both lateral and axial resolution characteristics
- Proper sampling (Nyquist criterion) is automatically validated
- The factory pattern ensures consistent instrument creation and validation